##Parte 4: Insights y Recomendaciones

##4.1 Análisis de Negocio

_Recordemos que se ha aplicado una tasa de conversión de 1 USD = 1 VES, dado que ambos indicadores presentan un comportamiento similar. Este ajuste permite simplificar el análisis sin distorsionar los resultados totales de la prueba_

**4.1.1.Desempeño General Q3 2025**
- El mes con mayor desempeño en ventas fue **julio** con un total de 108 MM. El margen bruto de la compañía del Q3 para el 2025 es en promedio 287,38 (USD o VES). El día de la semana que generó más ventas fue el **martes**.
Estos hallazgos permiten enfocar estrategias comerciales y operativas durante julio y optimizar recursos en los días de mayor actividad como el martes, aprovechando al máximo la rentabilidad reflejada en el margen bruto.

In [0]:
%sql
---------------------------------------------------
-- ¿Cuál fue el mes con mejor desempeño en ventas?: El mes de julio fue el mejor mes tanto en USD como en VES
---------------------------------------------------
-- 1. Seleccionamos moneda, periodo y sumamos las ventas
SELECT 
    CURRENCY,
    YEAR,
    MONTH,
    ROUND(SUM(TOTAL_SALE_TAX)/1000000,2) AS TOTAL_VENTAS
FROM workspace.operations.gold_sales_daily_summary
-- 2. Agrupamos por las primeras 3 columnas (Moneda, Año, Mes)
GROUP BY 1, 2, 3
-- 3. Filtramos para dejar solo el top 1 de ventas por cada moneda
QUALIFY ROW_NUMBER() OVER (PARTITION BY CURRENCY ORDER BY TOTAL_VENTAS DESC) = 1

In [0]:
%sql
---------------------------------------------------
-- ¿Cuál es el margen bruto promedio de la compañía?: 287.38 (USD o VES)
---------------------------------------------------
SELECT 
    ROUND(AVG(GROSS_MARGIN), 2) AS MARGEN_PROMEDIO
FROM workspace.operations.gold_sales_daily_summary


In [0]:
%sql
--------------------------------------------------------
-- ¿Qué día de la semana genera más ventas?: Martes
---------------------------------------------------------
-- 1. Seleccionamos el día de la semana y sumamos sus ventas
SELECT 
    DAY_OF_WEEK,
    SUM(TOTAL_SALE_TAX)/1000000 AS TOTAL_VENTAS
FROM workspace.operations.gold_sales_daily_summary
-- 2. Agrupamos los datos por día (Lunes, Martes, etc.)
GROUP BY DAY_OF_WEEK
-- 3. Ordenamos de mayor a menor venta
ORDER BY TOTAL_VENTAS DESC
-- 4. Nos quedamos solo con el día que más vendió
LIMIT 1;


**4.1.2 Análisis por Producto**
- En el Q3 2025, el departamento de **Nutrición** generó el mayor margen bruto con 33,28 MM, mientras que la marca **Genérico** lideró ventas con 54,99 MM.
Los productos de menor rotación fueron _Aun Cremas Genérico, Política Vitaminas Genérico y A Antibióticos Advanced_, con menos de 1.530 unidades cada uno.
Estos hallazgos indican oportunidades para potenciar los segmentos más rentables y optimizar el portafolio de baja rotación.

In [0]:
%sql
-- =========================================
-- Departamento con mayor margen bruto: Nutricion
-- =========================================
SELECT 
    PRODUCT_DEPARTMENT,  -- Nombre del departamento
    SUM(GROSS_MARGIN)/1000000 AS Margen-- Margen bruto 
FROM workspace.operations.gold_sales_daily_summary
GROUP BY PRODUCT_DEPARTMENT  -- Agrupa por departamento
ORDER BY Margen DESC       -- Ordena de mayor a menor margen
LIMIT 1;                       -- Retorna solo el departamento con mayor margen


In [0]:
%sql
-- =========================================
-- Marca con mayor volumen de ventas: Generico
-- Suma las ventas totales por marca y retorna la marca líder
-- =========================================
SELECT 
    PRODUCT_BRAND,             -- Nombre de la marca
    SUM(TOTAL_SALE_TAX)/1000000 AS VENTAS  -- Suma total de ventas (USD+VES, tasa=1)
FROM workspace.operations.gold_sales_daily_summary
GROUP BY PRODUCT_BRAND          -- Agrupa por marca
ORDER BY VENTAS DESC            -- Ordena de mayor a menor ventas
LIMIT 1;                        -- Retorna la marca con mayor ventas


In [0]:
%sql
-- =========================================
-- 3 productos con menor rotación
-- Suma las unidades vendidas por producto y retorna los 3 con menor volumen
-- =========================================
SELECT 
    PRODUCT_DESC,              -- Nombre del producto
    SUM(TOTAL_UNITS) AS UNIDADES_VENDIDAS  -- Total de unidades vendidas
FROM workspace.operations.gold_sales_daily_summary
GROUP BY PRODUCT_DESC          -- Agrupa por producto
ORDER BY UNIDADES_VENDIDAS ASC -- Ordena de menor a mayor unidades vendidas
LIMIT 3;                       -- Retorna los 3 productos con menor rotación


**4.1.3 Análisis Geográfico**
- La región **Centro** lidera tanto en ventas (129,06 MM) como en margen (50 MM), destacándose como el motor principal del negocio.
Las 5 tiendas con mejor desempeño se encuentran distribuidas entre Oriente, Capital y Occidente, con ventas cercanas a 3,5 MM cada una, indicando fuerte concentración de ingresos en ubicaciones estratégicas.
El rendimiento por formato de tienda es **equilibrado**; aunque Super lidera, Express logra facturar casi el 88% de su nivel, mostrando eficiencia y una distribución saludable de ingresos entre formatos.

In [0]:
%sql
-- =========================================
-- Región con más ventas 
-- La región centro con ventas de 129.06 MM
-- Calcula las ventas totales y el margen bruto por región
-- =========================================
SELECT 
    STORE_REGION,                     -- Nombre de la región
    SUM(TOTAL_SALE_TAX)/1000000 AS VENTAS     -- Suma total de ventas (USD+VES, tasa=1)
FROM workspace.operations.gold_sales_daily_summary
GROUP BY STORE_REGION                 -- Agrupa por región
ORDER BY VENTAS DESC                  -- Ordena de mayor a menor ventas
LIMIT 1;                              -- Retorna la región con mayor ventas



In [0]:
%sql
-- =========================================
-- Región mejor margen
-- La región centro con un margen de 50 MM
-- Calcula las ventas totales y el margen bruto por región
-- =========================================
SELECT 
    STORE_REGION,                     -- Nombre de la región
    SUM(GROSS_MARGIN)/1000000 AS Margen     -- Margen
FROM workspace.operations.gold_sales_daily_summary
GROUP BY STORE_REGION                 -- Agrupa por región
ORDER BY Margen DESC                  -- Ordena de mayor a menor 
LIMIT 1;                              -- Retorna la región con mayor 

Las 5 tiendas con mejor desempeño en ventas fueron:

- Farmatodo Maturin 45 
- Farmatodo Maturin 92 
- Farmatodo Caracas 61 
- Farmatodo Maracaibo 77 
- Farmatodo Barcelona 58 

In [0]:
%sql
-- =========================================
-- Top 5 tiendas con mejor desempeño
-- Calcula ventas totales por tienda y retorna las 5 principales
-- =========================================
SELECT 
    STORE_NAME,                      -- Nombre de la tienda
    STORE_REGION,                    -- Region
    SUM(TOTAL_SALE_TAX)/1000000 AS VENTAS    -- Suma de ventas totales
FROM workspace.operations.gold_sales_daily_summary
GROUP BY STORE_NAME, STORE_REGION               -- Agrupa por tienda
ORDER BY VENTAS DESC                 -- Ordena de mayor a menor ventas
LIMIT 5;                             -- Retorna las 5 tiendas con mayor venta


Diferencia entre formatos de tienda
- No existe una diferencia significativa
- Existe una diferencia de solo el 13% entre el formato líder (Super) y el menor (Express)
- El formato Express destaca por su competitividad, logrando facturar casi el 88% de lo que genera un Super, a pesar de tener (presuntamente) menor superficie de venta.

In [0]:
%sql
-- =========================================
-- Comparación de ventas por formato de tienda
-- Evalúa si existen diferencias significativas entre los formatos (ej: Express, Mega, Super)
-- =========================================
SELECT 
    STORE_FORMAT,                     -- Tipo de formato de la tienda
    SUM(TOTAL_SALE_TAX)/1000000 AS VENTAS     -- Suma total de ventas
FROM workspace.operations.gold_sales_daily_summary
GROUP BY STORE_FORMAT                -- Agrupa por formato
ORDER BY VENTAS DESC;                -- Ordena de mayor a menor ventas


**4.1.4 Análisis de Canales**
- El 49,86% de las ventas proviene de canales digitales (E-Commerce + App), mostrando una fuerte adopción de los canales online. El ticket promedio es muy similar entre todos los canales, con Tienda Física en 739,62, lo que indica consistencia en el gasto promedio del cliente.
El canal más rentable es la **Tienda Física**, con un margen total de 31,15 MM, destacando su importancia estratégica para la rentabilidad.

In [0]:
%sql
-- =========================================
-- Porcentaje de ventas provenientes de canales digitales
-- Suma las ventas de E-Commerce y App y calcula su proporción sobre el total
-- =========================================
WITH ventas_digitales AS (
    SELECT SUM(TOTAL_SALE_TAX) AS VENTAS_DIGITAL
    FROM workspace.operations.gold_sales_daily_summary
    WHERE CHANNEL_NAME IN ('E-Commerce','App Movil')  -- Filtra solo canales digitales
),
ventas_totales AS (
    SELECT SUM(TOTAL_SALE_TAX) AS VENTAS_TOT
    FROM workspace.operations.gold_sales_daily_summary
)
SELECT 
    ROUND(v.VENTAS_DIGITAL / t.VENTAS_TOT * 100,2) AS PCT_VENTAS_DIGITAL
FROM ventas_digitales v
CROSS JOIN ventas_totales t;


In [0]:
%sql
-- =========================================
-- Ticket promedio por canal
-- Calcula el ticket promedio (ventas / transacciones) para cada canal
-- =========================================
SELECT 
    CHANNEL_NAME,  -- Nombre del canal
    ROUND(SUM(TOTAL_SALE_TAX) / SUM(TOTAL_TRANSACTIONS),2) AS AVG_TICKET
FROM workspace.operations.gold_sales_daily_summary
GROUP BY CHANNEL_NAME
ORDER BY AVG_TICKET DESC;


In [0]:
%sql
-- =========================================
-- Canal con mayor margen bruto
-- Calcula el margen bruto porcentual por canal y ordena de mayor a menor
-- =========================================
SELECT 
    CHANNEL_NAME,  
    ROUND(SUM(GROSS_MARGIN),2)/1000000 AS MARGEN
FROM workspace.operations.gold_sales_daily_summary
GROUP BY CHANNEL_NAME
ORDER BY MARGEN DESC
LIMIT 1;  -- Retorna el canal con mayor margen bruto


**4.1.5 Análisis de Proveedores**
- Los tres principales proveedores con mayores ventas fueron Pfizer Venezuela, Johnson & Johnson y Sanofi Venezuela, con ventas cercanas a 34 MM cada uno. Estos proveedores representan aproximadamente el 31,5% de las ventas totales, mostrando una concentración significativa en pocos socios estratégicos.
El análisis sugiere la importancia de mantener relaciones sólidas con estos proveedores y considerar estrategias de diversificación para reducir riesgos.



In [0]:
%sql
-- =========================================
-- Top 3 proveedores por volumen de ventas
-- Suma las ventas totales por proveedor y retorna los 3 mayores
-- =========================================
SELECT 
    SUPPLIER_NAME,                  -- Nombre del proveedor
    SUM(TOTAL_SALE_TAX)/1000000 AS VENTAS   -- Suma de ventas totales (USD+VES, tasa=1)
FROM workspace.operations.gold_sales_daily_summary
GROUP BY SUPPLIER_NAME             -- Agrupa por proveedor
ORDER BY VENTAS DESC               -- Ordena de mayor a menor ventas
LIMIT 3;                           -- Retorna los 3 proveedores principales


In [0]:
%sql
-- ========================================================
-- Porcentaje de ventas totales de los top 3 proveedores
-- ========================================================

--  Identificar quiénes son los 3 proveedores que más venden
WITH top3_proveedores AS (
    SELECT 
        SUPPLIER_NAME,
        SUM(TOTAL_SALE_TAX) AS VENTAS
    FROM workspace.operations.gold_sales_daily_summary
    GROUP BY SUPPLIER_NAME
    ORDER BY VENTAS DESC
    LIMIT 3
),

-- Calcular el monto total de ventas de toda la empresa 
total_ventas AS (
    SELECT SUM(TOTAL_SALE_TAX) AS VENTAS_TOTALES
    FROM workspace.operations.gold_sales_daily_summary
)

-- Unir ambos cálculos para obtener el porcentaje de participación
SELECT 
    -- Sumamos las ventas del Top 3 y las dividimos entre la venta global
    ROUND(SUM(t.VENTAS) OVER() / tot.VENTAS_TOTALES * 100, 2) AS PCT_TOP3
FROM top3_proveedores t
CROSS JOIN total_ventas tot
-- Limitamos a 1 para mostrar un único resultado final
LIMIT 1;

In [0]:
%sql
-------------------------------------
--- Porcentaje de venta por proveedor
------------------------------------

WITH ventas_por_proveedor AS (
    -- 1. Calculamos la venta total de cada proveedor
    SELECT 
        SUPPLIER_NAME,
        SUM(TOTAL_SALE_TAX) AS VENTAS_PROVEEDOR
    FROM workspace.operations.gold_sales_daily_summary
    GROUP BY ALL
),
total_global AS (
    -- 2. Obtenemos la venta total de toda la empresa
    SELECT SUM(TOTAL_SALE_TAX) AS VENTA_TOTAL_CIA FROM workspace.operations.gold_sales_daily_summary
)
SELECT 
    v.SUPPLIER_NAME,
    v.VENTAS_PROVEEDOR,
    -- 3. Calculamos el % de participación de cada uno sobre el total
    ROUND((v.VENTAS_PROVEEDOR / tg.VENTA_TOTAL_CIA) * 100, 2) AS PORCENTAJE_PARTICIPACION
FROM ventas_por_proveedor v
CROSS JOIN total_global tg
ORDER BY v.VENTAS_PROVEEDOR DESC
LIMIT 3;

**4.1.6 Análisis Monetario**
- El 70,1% de las transacciones se realizaron en bolívares (VES), mientras que el 29,9% corresponde a dólares (USD). El ticket promedio es similar entre ambas monedas: 737,9 VES vs 736,2 USD.

In [0]:
%sql
-- ========================================================
-- Análisis de participación por moneda en transacciones
-- ========================================================

-- Contar cuántas transacciones existen por cada moneda
WITH transacciones_por_moneda AS (
    SELECT 
        CURRENCY, 
        COUNT(*) AS TRANSACCIONES  
    FROM workspace.operations.gold_sales_daily_summary
    GROUP BY CURRENCY
),

--  Obtener el conteo total de transacciones en toda la tabla
total_transacciones AS (
    SELECT COUNT(*) AS TOTAL_TRANS
    FROM workspace.operations.gold_sales_daily_summary
)

--  Unir los datos para calcular el peso de cada moneda sobre el total
SELECT 
    t.CURRENCY,
    t.TRANSACCIONES,
    -- Dividimos las transacciones de la moneda entre el total global y multiplicamos por 100
    ROUND(t.TRANSACCIONES / tot.TOTAL_TRANS * 100, 2) AS PCT_TRANSACCIONES
FROM transacciones_por_moneda t
-- El CROSS JOIN nos permite comparar cada fila de moneda contra el único valor del total global
CROSS JOIN total_transacciones tot;

In [0]:
%sql
-- =========================================
-- Ticket promedio por moneda
-- Calcula el ticket promedio (TOTAL_SALE_TAX / TOTAL_TRANSACTIONS) para USD y VES
-- =========================================
SELECT 
    CURRENCY,                                 -- Moneda de la transacción
    ROUND(SUM(TOTAL_SALE_TAX) / SUM(TOTAL_TRANSACTIONS), 2) AS AVG_TICKET  -- Ticket promedio
FROM workspace.operations.gold_sales_daily_summary
GROUP BY CURRENCY
ORDER BY AVG_TICKET DESC;


**Análisis adicionales**

**Análisis de tendencia de ventas**

In [0]:
%sql
SELECT
    -- 1. Normalizamos la fecha al primer día del mes
    DATE_TRUNC('MONTH', BUSINESS_DATE) AS month,
    
    -- 2. Cálculos financieros en millones con redondeo para mejor lectura
    ROUND(SUM(TOTAL_SALE_TAX) / 1000000, 2) AS monthly_sales_m,
    ROUND(SUM(GROSS_MARGIN) / 1000000, 2) AS monthly_margin_m,
    
    -- 3. Métricas operativas
    SUM(TOTAL_TRANSACTIONS) AS monthly_transactions,
    
    -- 4. KPI extra: Ticket promedio (Venta / Transacciones)
    ROUND(SUM(TOTAL_SALE_TAX) / SUM(TOTAL_TRANSACTIONS), 2) AS avg_ticket
FROM
    workspace.operations.gold_sales_daily_summary
WHERE
    -- Filtramos nulos de forma más eficiente
    BUSINESS_DATE IS NOT NULL
    AND TOTAL_SALE_TAX > 0 
GROUP BY 1
ORDER BY
    month ASC;

**Productos estrellas**

Los productos más vendidos que también generaron mayor ganancia son "Todos Accesorios Premium", "Blanco Snacks Advanced" y "Reunión Analgesicos Essential", cada uno con más de 1,900 unidades vendidas y un margen bruto superior a 184,000. Esto indica que los productos con mayor volumen de ventas también aportan significativamente al margen bruto

In [0]:
%sql
-- ========================================================
-- Búsqueda de Productos Estrella (Alto Volumen + Alto Margen)
-- ========================================================
SELECT
    PRODUCT_DESC,
    SUM(TOTAL_UNITS) AS total_unidades,
    ROUND(SUM(GROSS_MARGIN), 2) AS margen_total,
    -- KPI: Margen promedio por unidad vendida
    ROUND(SUM(GROSS_MARGIN) / SUM(TOTAL_UNITS), 2) AS margen_por_unidad
FROM
    workspace.operations.gold_sales_daily_summary
WHERE
    -- Limpiamos datos para asegurar que tengan unidades y margen positivo
    TOTAL_UNITS > 0 
    AND GROSS_MARGIN > 0
GROUP BY 
    PRODUCT_DESC
-- El secreto: Ordenamos por las dos métricas clave al mismo tiempo
-- Esto prioriza los que tienen números altos en ambos campos
ORDER BY 
    total_unidades DESC, 
    margen_total DESC
LIMIT 3;

**Ciudades con mayor y menor desempeño y su canal principal**

In [0]:
%sql
-- ========================================================
-- Ciudades con mayor y menor desempeño y su canal principal
-- ========================================================
WITH ventas_por_ciudad_canal AS (
    -- 1. Agrupamos ventas por ciudad y canal
    SELECT 
        STORE_CITY,
        CHANNEL_NAME,
        SUM(TOTAL_SALE_TAX) AS ventas_totales
    FROM workspace.operations.gold_sales_daily_summary
    GROUP BY 1, 2
),
ranking_ciudades AS (
    -- 2. Identificamos las ciudades extremas (Top 1 y Bottom 1)
    SELECT 
        STORE_CITY,
        SUM(ventas_totales) AS ventas_ciudad,
        -- Asignamos posición para extraer los extremos fácilmente
        ROW_NUMBER() OVER (ORDER BY SUM(ventas_totales) DESC) as rank_max,
        ROW_NUMBER() OVER (ORDER BY SUM(ventas_totales) ASC) as rank_min
    FROM ventas_por_ciudad_canal
    GROUP BY 1
),
extremos AS (
    -- 3. Filtramos solo la mejor y la peor ciudad
    SELECT STORE_CITY, ventas_ciudad, 'Máxima Venta' as tipo FROM ranking_ciudades WHERE rank_max = 1
    UNION ALL
    SELECT STORE_CITY, ventas_ciudad, 'Mínima Venta' as tipo FROM ranking_ciudades WHERE rank_min = 1
)
-- 4. Resultado final: Ciudad, Venta Total y el canal más usado en esa ciudad
SELECT 
    e.tipo,
    e.STORE_CITY AS ciudad,
    ROUND(e.ventas_ciudad / 1000000, 2) AS ventas_millones,
    (
        SELECT v.channel_name 
        FROM ventas_por_ciudad_canal v 
        WHERE v.STORE_CITY = e.STORE_CITY 
        ORDER BY v.ventas_totales DESC LIMIT 1
    ) AS canal_predominante
FROM extremos e;

**Productos mas y menos vendidos por ciudad**

In [0]:
%sql
-- ========================================================
-- Producto más y menos vendido por ciudad
-- ========================================================
WITH city_product_units AS (
  SELECT
    STORE_CITY,
    PRODUCT_CLASS,
    SUM(TOTAL_UNITS) AS total_units
  FROM
    workspace.operations.gold_sales_daily_summary
  WHERE
    -- Simplificamos filtros: si total_units es > 0, los descriptivos suelen estar
    STORE_CITY IS NOT NULL 
    AND TOTAL_UNITS > 0
  GROUP BY 
    1, 2
),
ranked AS (
  SELECT
    *,
    -- Usamos RANK() en lugar de ROW_NUMBER() por si hay empates en ventas
    RANK() OVER (PARTITION BY STORE_CITY ORDER BY total_units DESC) AS rn_max,
    RANK() OVER (PARTITION BY STORE_CITY ORDER BY total_units ASC) AS rn_min
  FROM
    city_product_units
)
SELECT
  STORE_CITY,
  product_class,
  total_units,
  CASE WHEN rn_max = 1 THEN 'Más Vendido' ELSE 'Menos Vendido' END AS tipo
FROM
  ranked
WHERE
  rn_max = 1 OR rn_min = 1
ORDER BY
  STORE_CITY, 
  total_units DESC;

In [0]:
%sql
-- ========================================================
-- Producto con mayor contribución a las ventas en Q3 2025
-- Optimizado con Funciones de Ventana para mayor eficiencia
-- ========================================================
WITH sales_summary AS (
    SELECT
        PRODUCT_DESC,
        SUM(TOTAL_SALE_TAX) AS total_sale_tax,
        -- Calculamos el gran total de todo el periodo
        SUM(SUM(TOTAL_SALE_TAX)) OVER() AS grand_total
    FROM
        workspace.operations.gold_sales_daily_summary
    GROUP BY
        1
)
SELECT
    PRODUCT_DESC,
    ROUND(total_sale_tax, 2) AS total_sale_tax,
    -- Calculamos el porcentaje de contribución directamente
    ROUND(100.0 * total_sale_tax / grand_total, 2) AS pct_contribution
FROM
    sales_summary
ORDER BY
    total_sale_tax DESC
LIMIT 1;

## Insights – Q3 2025

**Sección 1: Resumen Ejecutivo**
1. Durante el Q3 2025, el negocio generó **319,3 MM en ventas**, con un **margen bruto de 123,8 MM** y **433 mil transacciones**, evidenciando un trimestre sólido en volumen, aunque con señales claras de **desaceleración progresiva** hacia el cierre del período.  

2. Julio concentró el **mejor desempeño del trimestre** en ventas, margen y número de transacciones, mientras que septiembre registró los valores más bajos (**104 MM en ventas y 141.372 transacciones**), lo que indica una **caída mes a mes en la actividad comercial**.  

3. Los **canales digitales** (E-Commerce + App) representan **49,86% de las ventas totales**, mostrando una adopción significativa por parte del cliente; sin embargo, la **Tienda Física continúa siendo el canal más rentable**, con **31,15 MM en margen bruto**, lo que confirma su rol clave en la rentabilidad del negocio.  

4. Los productos con mayor volumen de ventas también son los que generan **mayor margen**, lo que sugiere una **alineación saludable entre volumen y rentabilidad**, reduciendo el riesgo de crecimiento no rentable.

---

**Sección 2: Hallazgos Detallados**

**Productos**
- El **departamento de Nutrición** lidera el margen bruto del trimestre con **33,28 MM**, mientras que la **marca Genérico** concentró el mayor nivel de ventas (**54,99 MM**), confirmando su peso estratégico en el portafolio.  
- El producto **“Oficial Bebidas Natural”** fue el de mayor venta, aportando aproximadamente **27% de las ventas del trimestre**, consolidándose como un producto ancla para el negocio.  
- Se identificaron productos de **baja rotación** (*Aun Cremas Genérico*, *Política Vitaminas Genérico* y *A Antibióticos Advanced*), todos con menos de **1.530 unidades vendidas**, lo que evidencia oportunidades de **optimización de inventario**.  
- Existe una **especialización regional clara**: Caracas y Barquisimeto se enfocan en **Suplementos**, mientras que Maracaibo y Barcelona concentran consumo familiar (**Fórmulas**). El bajo desempeño de **Pañales y Analgésicos** sugiere oportunidades de **cross-selling**, ya que el cliente podría estar completando estas compras en la competencia.

**Geografía**
- La **región Centro** lidera el desempeño del negocio, concentrando **129,06 MM en ventas** y **50 MM en margen**, posicionándose como la región más rentable del trimestre.  
- A nivel de ciudades, **Maracaibo** registra la mayor venta (**60,66 MM**), mientras que **Barcelona** presenta el menor desempeño (**27,48 MM**), lo que refleja disparidades claras en el potencial comercial por ciudad.  
- Las tiendas con mejor desempeño incluyen **Farmatodo Maturín 45 y 92, Caracas 61, Maracaibo 77 y Barcelona 58**, que pueden considerarse referentes para replicar buenas prácticas.  
- El formato **Express** factura aproximadamente **88% de un formato Super**, evidenciando **alta eficiencia operativa** pese a su menor tamaño.

**Canales**
- Los canales digitales concentran **49,86% de las ventas**, confirmando un cambio estructural en el comportamiento del consumidor.  
- El **ticket promedio es consistente** entre canales (~737), lo que indica que el cliente mantiene su nivel de gasto independientemente del canal utilizado.  
- La **Tienda Física** genera el mayor margen bruto (**31,15 MM**), reforzando su importancia en la estrategia de rentabilidad.

**Proveedores**
- Los tres principales proveedores (**Pfizer Venezuela, Johnson & Johnson y Sanofi Venezuela**) concentran **31,5% de las ventas**, lo que evidencia una **alta dependencia** en un grupo reducido de proveedores estratégicos.

**Moneda**
- El **70% de las transacciones se realizan en VES** y el **30% en USD**, con tickets promedio prácticamente iguales (**737,9 VES vs 736,2 USD**), lo que indica **neutralidad de la moneda** en el comportamiento de compra del cliente.

---

**Sección 3: Oportunidades Identificadas**

1. **Optimización del portafolio de baja rotación**  
   Productos con menos de **1.530 unidades vendidas** presentan baja contribución al negocio, representando inventario inmovilizado y costos operativos innecesarios.

2. **Crecimiento de canales digitales**  
   Con **casi el 50% de las ventas** ya digitalizadas, existe una oportunidad clara de escalar estos canales mediante mejoras en experiencia de usuario y campañas dirigidas.

3. **Cross-selling regional por patrón de consumo**  
   El bajo volumen de Pañales y Analgésicos en varias ciudades sugiere compras incompletas, abriendo espacio para incrementar ticket promedio mediante ventas complementarias.

4. **Mayor explotación de la región Centro**  
   Al concentrar el mayor volumen de ventas y margen, la región Centro ofrece el **mayor retorno potencial** para inversiones comerciales y operativas.

5. **Diversificación de proveedores estratégicos**  
   La concentración del 31,5% de las ventas en tres proveedores incrementa el riesgo operativo y limita el poder de negociación.

---

**Sección 4: Recomendaciones Accionables (Priorizadas por Impacto)**

📌 **Recomendación:** Potenciar productos de alta rentabilidad en Nutrición  
📊 **Insight:** Nutrición lidera el margen bruto con **33,28 MM**  
🎯 **Impacto esperado:** Alto  
⚡ **Acción:** Implementar campañas quincenales enfocadas en productos de Nutrición en tiendas físicas y canales digitales; responsables: Marketing y Ventas.  
**Justificación:** Maximiza retorno inmediato al priorizar productos ya rentables.

---

📌 **Recomendación:** Escalar el crecimiento de canales digitales  
📊 **Insight:** Canales digitales representan **49,86% de las ventas**  
🎯 **Impacto esperado:** Alto  
⚡ **Acción:** Optimizar UX de App y E-Commerce e implementar promociones exclusivas online; responsables: Marketing Digital y TI.  
**Justificación:** Incrementa alcance y reduce dependencia del tráfico físico.

---

📌 **Recomendación:** Focalizar inversión comercial en la región Centro  
📊 **Insight:** Centro concentra **129,06 MM en ventas** y **50 MM en margen**  
🎯 **Impacto esperado:** Alto  
⚡ **Acción:** Ajustar surtido, promociones y evaluar nuevas aperturas; responsables: Ventas y Operaciones Regionales.  
**Justificación:** Prioriza recursos donde el retorno es mayor.

---

📌 **Recomendación:** Optimizar inventario de baja rotación  
📊 **Insight:** Productos con menos de **1.530 unidades vendidas**  
🎯 **Impacto esperado:** Medio  
⚡ **Acción:** Redistribuir, promocionar o descontinuar productos; responsables: Compras y Operaciones.  
**Justificación:** Reduce costos y libera capital de trabajo.

---

📌 **Recomendación:** Implementar estrategias de cross-selling regional  
📊 **Insight:** Bajo desempeño de Pañales y Analgésicos en varias ciudades  
🎯 **Impacto esperado:** Medio  
⚡ **Acción:** Crear bundles y promociones combinadas por ciudad; responsables: Marketing y Ventas Regionales.  
**Justificación:** Aumenta ticket promedio y fidelización.

---

📌 **Recomendación:** Diversificar proveedores estratégicos  
📊 **Insight:** Top 3 proveedores concentran **31,5% de las ventas**  
🎯 **Impacto esperado:** Medio  
⚡ **Acción:** Incorporar proveedores alternativos y renegociar contratos; responsables: Compras.  
**Justificación:** Reduce riesgo operativo y mejora poder de negociación.

---

📌 **Recomendación:** Activar promociones para fines de semana  
📊 **Insight:** Martes concentra mayor actividad; domingos presentan menor volumen  
🎯 **Impacto esperado:** Medio  
⚡ **Acción:** Implementar promociones y campañas específicas viernes–domingo; responsables: Marketing y Operaciones.  
**Justificación:** Aprovecha capacidad comercial subutilizada y equilibra la demanda semanal.
